<a href="https://colab.research.google.com/github/NataliaKolesnik/Car_Price_prediction_part2/blob/master/2_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 


In [3]:
path_data = '/content/drive/MyDrive/SF_project_8/data/'

##Библиотеки

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

from catboost import Pool, CatBoostRegressor

In [5]:
pd.set_option('display.float_format', '{:.2f}'.format)

SEED = 42

## Загрузка и подготовка данных для обучения

In [6]:
df_train = pd.read_csv(path_data+'train_for_ML.csv')
df_test = pd.read_csv(path_data+'test_for_ML.csv')

In [7]:
# Списки с признаками

categorical_features = ['bodyType', 'brand', 'color_littel', 'fuelType_littel', 'model_info',
                        'vehicleTransmission', 'ПТС', 'Привод', ]

numerical_features = ['mileage', 'age_avto', 'engineDisplacement', 'enginePower_log', 'numberOfDoors', 'Владельцы']

target = 'price'
target_log = 'price_log'

In [8]:
# Соберем датасеты

X = df_train[categorical_features + numerical_features]

X_test = df_test[categorical_features + numerical_features]

Y = df_train[target]
Y_log = df_train[target_log]

In [9]:
# Стандартизируем числовые данные

scaler = MinMaxScaler()

X[numerical_features] = scaler.fit_transform(X[numerical_features])
X_test[numerical_features] = scaler.fit_transform(X_test[numerical_features])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [10]:
X.describe()

,mileage,age_avto,engineDisplacement,enginePower_log,numberOfDoors,Владельцы
count,6682.00,6682.00,6682.00,6682.00,6682.00,6682.00
mean,0.12,0.21,0.42,0.54,0.76,0.55
std,0.08,0.13,0.16,0.18,0.29,0.42
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.06,0.11,0.30,0.42,0.67,0.00
50%,0.11,0.18,0.42,0.53,0.67,0.50
75%,0.17,0.29,0.45,0.66,1.00,1.00
max,1.00,1.00,1.00,1.00,1.00,1.00


In [11]:
X_test.describe()

,mileage,age_avto,engineDisplacement,enginePower_log,numberOfDoors,Владельцы
count,1671.00,1671.00,1671.00,1671.00,1671.00,1671.00
mean,0.12,0.21,0.42,0.44,0.75,0.55
std,0.08,0.14,0.16,0.19,0.30,0.41
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.06,0.11,0.30,0.31,0.67,0.00
50%,0.11,0.18,0.41,0.43,0.67,0.50
75%,0.17,0.29,0.45,0.57,1.00,1.00
max,1.00,1.00,1.00,1.00,1.00,1.00


In [12]:
# Разбиваем на обучающую и валидационную выборки

X_train, X_valid, Ylog_train, Ylog_valid = train_test_split(X, Y_log, test_size=0.2, random_state=SEED, shuffle=True)
X_train_, X_valid_, Y_train, Y_valid = train_test_split(X, Y, test_size=0.2, random_state=SEED, shuffle=True)

In [13]:
# Готовим pool для catboost

train_pool = Pool(X_train, Y_train, cat_features=categorical_features)
valid_pool = Pool(X_valid, Y_valid, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

## Обучение модели

In [14]:
# создаем модель
model = CatBoostRegressor(
    depth=8, 
    l2_leaf_reg=3,
    learning_rate=0.03, #0.03
    iterations=10000, 
    eval_metric='MAPE', 
    custom_metric=['RMSE', 'MAE'],
    loss_function= 'RMSE', #'MAE', 
    random_seed=SEED,
    # grow_policy='Depthwise',
    # score_function='L2'
    od_wait=500
    )


In [15]:
# обучаем модель
model.fit(train_pool, 
          eval_set=valid_pool, 
          #logging_level='Silent', 
          verbose_eval=500,
          use_best_model=True,
          plot=False)

0:	learn: 0.9749677	test: 1.0081246	best: 1.0081246 (0)	total: 77.2ms	remaining: 12m 51s
500:	learn: 0.1447294	test: 0.1576494	best: 0.1576385 (499)	total: 19.8s	remaining: 6m 14s
1000:	learn: 0.1207464	test: 0.1458115	best: 0.1458115 (1000)	total: 33.7s	remaining: 5m 3s
1500:	learn: 0.1057603	test: 0.1393638	best: 0.1393638 (1500)	total: 44.1s	remaining: 4m 9s
2000:	learn: 0.0951163	test: 0.1372988	best: 0.1372849 (1998)	total: 54.5s	remaining: 3m 37s
2500:	learn: 0.0873558	test: 0.1363858	best: 0.1363013 (2318)	total: 1m 7s	remaining: 3m 21s
3000:	learn: 0.0805190	test: 0.1359304	best: 0.1357733 (2942)	total: 1m 17s	remaining: 3m
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.1357732583
bestIteration = 2942

Shrink model to first 2943 iterations.


In [16]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,mileage,21.64
1,age_avto,19.99
2,enginePower_log,17.17
3,engineDisplacement,11.81
4,brand,6.60
5,bodyType,5.29
6,model_info,4.85
7,Привод,4.06
8,fuelType_littel,3.19
9,color_littel,2.67


In [17]:
# model_gp = CatBoostRegressor(
#     iterations=1000, 
#     eval_metric='MAPE', 
#     )

# grid = {
#         'learning_rate': [0.03, 0.01, 0.003, 0.008],
#         'depth': [2, 4, 6, 8],
#         'l2_leaf_reg': [0.2, 0.5, 1, 3],
#         'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
#         'score_function': ['Cosine', 'L2', 'NewtonL2', 'NewtonCosine'],
#         }
# model_gp.grid_search(grid, train_pool, verbose=200)

In [18]:
# Проверка предсказания на вилидационных данных
y_valid_predict = model.predict(valid_pool)
#y_valid_predict = np.exp(y_valid_predict) 
mean_absolute_percentage_error(Y_valid, y_valid_predict)

0.13577325834737236

## Предсказание

In [19]:
# предсказываем результат 
y_test_predict = model.predict(test_pool)

In [20]:
y_test_predict

array([ 882457.76389735, 2297140.95591029,  639638.87310253, ...,
       1510527.96144637, 1030386.33154869, 1042252.21969542])

In [21]:
#y_test_predict = np.exp(y_test_predict)

In [22]:
submission = pd.read_csv(path_data+'sample_submission.csv')
submission['price'] = y_test_predict
submission.to_csv(path_data+'sample_submission_catboost.csv', index=False)

In [23]:
submission

,sell_id,price
0,1099427284,882457.76
1,1096405886,2297140.96
2,1100195294,639638.87
3,1099827282,445014.54
4,1100076198,2242855.78
...,...,...
1666,1099414488,882010.04
1667,1099277058,2516290.93
1668,1098773964,1510527.96
1669,1098835932,1030386.33
